# **Rainfall API of SG Data Portal**
To get the rainfall across Singapore using the API, retrieving from JSON format. 
This API will let users fetch rainfall readings across Singapore:
* 5-minute readings from NEA
* Filter for specific date or date-time by providing date in query parameter by using:
  - use YYYY-MM-DD format to retrieve all of the readings for that day
  - use YYYY-MM-DDTHH:mm:ss to retrieve the latest readings at that moment in time
  - example: ?date=2024-07-16 or ?date=2024-07-16T23:59:00

# **Prerequisites**
Install the Python libraries on your local machine:
1. requests for making API calls
2. pandas for handling and manipulating data
3. pytz for handling timezones
4. csv library for writing data to CSV files

Skip these steps if you have installed all the libraries.

In [ ]:
# Open a new terminal and run the following command:
pip install requests 
pip install pandas
pip install pytz

In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os

def get_historical_rainfall_readings():
    # Define the API URL
    api_url = "https://api.data.gov.sg/v1/environment/rainfall"

    # Calculate the date range 
    start_date = datetime(2024, 10, 1)  # Example start date
    end_date = datetime(2024, 10, 1)    # Example end date (same for this case)
    
    # Collect all readings in a list
    all_readings = []

    # Iterate through each day from start_date to end_date
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')  # Format date as YYYY-MM-DD
        
        # Parameters for the API request
        params = {
            'date': date_str  # Use the correct date parameter
        }

        # Make the GET request
        response = requests.get(api_url, params=params)

        # Check successful response
        if response.status_code == 200:
            data = response.json()
            
            # Check if 'items' are present in the response
            if "items" in data:
                for item in data["items"]:
                    timestamp = item["timestamp"]
                    for reading in item["readings"]:
                        station_id = reading["station_id"]
                        rainfall_value = reading["value"]
                        all_readings.append({
                            "Timestamp": timestamp,
                            "Station ID": station_id,
                            "Rainfall (mm)": rainfall_value
                        })
            else:
                print(f"No readings found for {date_str}.")
        else:
            print(f"Failed to retrieve data for {date_str}: {response.status_code} - {response.text}")

        # Move to the next day
        current_date += timedelta(days=1)

        # Optional: Sleep for a second to avoid hitting rate limits
        time.sleep(1)

    # Create a DataFrame from the collected readings
    rainfall_df = pd.DataFrame(all_readings)

    # Print the DataFrame for user to see
    if not rainfall_df.empty:
        print("\nHistorical Rainfall Data Table:")
        print(rainfall_df)
        
        # Prompt the user to save data to CSV
        save_choice = input("Do you want to save this data to a CSV file? (yes/no): ").strip().lower()
        
        if save_choice == 'yes':
            # Specify the output directory
            output_dir = '/Users/yvonne/Downloads'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Get the current date and time for the filename
            current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
            rainfall_csv_file = os.path.join(output_dir, f'rainfall_data_{current_time}.csv')

            # Write rainfall data to CSV
            rainfall_df.to_csv(rainfall_csv_file, index=False)
            print(f"Rainfall data has been successfully exported to {rainfall_csv_file}")
        else:
            print("Data not saved.")
    else:
        print("No rainfall data available for the given date range.")

# Call the function to get historical rainfall readings
get_historical_rainfall_readings()


Historical Rainfall Data Table:
                       Timestamp Station ID  Rainfall (mm)
0      2024-10-01T00:05:00+08:00       S224            0.0
1      2024-10-01T00:05:00+08:00        S77            0.0
2      2024-10-01T00:05:00+08:00       S216            0.0
3      2024-10-01T00:05:00+08:00       S109            0.0
4      2024-10-01T00:05:00+08:00       S117            0.0
...                          ...        ...            ...
17319  2024-10-01T23:55:00+08:00       S104            0.0
17320  2024-10-01T23:55:00+08:00       S210            0.0
17321  2024-10-01T23:55:00+08:00       S227            0.0
17322  2024-10-01T23:55:00+08:00       S219            0.0
17323  2024-10-01T23:55:00+08:00       S209            0.0

[17324 rows x 3 columns]
Data not saved.
